In [2782]:
import importlib
import os
import sys

import pandas as pd

import matplotlib.pyplot as plt
import plotly.graph_objects as go

from datetime import datetime

from plotly.subplots import make_subplots

In [2783]:
# Makes sure modules are available to the notebook that acts as an entrypoint

parent_dir = os.path.abspath('..')

if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [2784]:
# input_file = 'plug-and-play.csv'
# input_file = 'plug-and-play-2023-05-24-tr_first_then_roll.csv'
input_file = 'SPY:CustomLongStrategy.csv'

df = pd.read_csv(os.getcwd() + f'/../outputs/{input_file}')

print(df['action'].value_counts())

print(df[df['action'] == 1.0])

df.set_index('datetime', inplace=True)
df = df.loc['2023-05-02 09:30':'2023-05-02 16:01']
df.reset_index(inplace=True)

# Long SkewKurtVol

# Kurt < 3.0 and Skew > 0 and Vol > vol_mean
# Win: 8  Loss: 7  53% win rate

# Kurt < 3.0 and Skew > 0 and Vol < vol_mean
# Win: 10  Loss: 6  62.5% win rate

# Kurt > 3.0 and Skew < 0 and Vol > vol_mean
# Win: 2  Loss: 3  40% win rate

# Kurt > 3.0 and Skew < 0 and Vol < vol_mean
# Win: 2  Loss: 2  50% win rate

####

# Kurt < 0 and Skew > 0 and Vol > vol_mean
# Win: 7  Loss: 7  50% win rate

# Kurt < 0 and Skew > 0 and Vol < vol_mean
# Win: 8  Loss: 4  66.6% win rate

# Kurt > 0 and Skew < 0 and Vol > vol_mean
# Win: 3  Loss: 6  33.3% win rate

# Kurt > 0 and Skew < 0 and Vol < vol_mean
# Win: 4  Loss: 5  44% win rate

####

# Kurt > 0 and Skew > 0 and Vol > vol_mean
# Win: 6  Loss: 4  60% win rate

# Kurt > 0 and Skew > 0 and Vol < vol_mean
# Win: 5  Loss: 5  50% win rate

# Kurt < 0 and Skew < 0 and Vol > vol_mean
# Win: 4  Loss: 8  33.3% win rate

# Kurt < 0 and Skew < 0 and Vol < vol_mean
# Win: 7  Loss: 5  58.3% win rate

###

# Long PriceChannels

# Win: 5  Loss: 1  83% win rate

# Long PriceChannels with Upper Band via TR
# Win: 7 Loss: 1  87.5% win rate

# print(df['open'].idxmax())
# print(df.iloc[df['open'].idxmax()])

# df.iloc[45 : 50]


0.0    8744
1.0       1
Name: action, dtype: int64
     index  action ticker             datetime    close    high     low  \
538   1876     1.0    SPY  2023-05-02 09:34:00  414.285  414.29  414.03   

       open  volume   sw  ...     u_bound         lbf        rsi    tr  \
538  414.14  245818  0.0  ...  415.571208  413.917753  42.300279  0.26   

          atr         avg      std      skew      kurt   z_score  
538  0.173459  415.987903  0.55337 -0.705072  2.421442 -3.077334  

[1 rows x 29 columns]


In [2785]:
# Or datetime
x=df.index.values

y1=df['close'].values
y2=df['action'].values

# Create the first trace with the primary y-axis
trace1 = go.Scatter(x=x, y=y1, name='Open')

# Create the second trace with the secondary y-axis
trace2 = go.Scatter(x=x, y=y2, name='Action', yaxis='y2')

# Label highest score
max_index = df['action'].idxmax()
highest_score = format(df['action'][max_index], ".4f")


# Create the layout with two y-axes
layout = go.Layout(
    title=f"{df['ticker'].values[0]} {df['datetime'].values[0]} - {df['datetime'].values[-1]}  Highest Score: {highest_score}",
    yaxis=dict(),
    yaxis2=dict(overlaying='y', side='right'),
    height=650
)

# layout = go.Layout(
#     title=f"{df['Ticker'].values[0]} {df['datetime'].values[0]} - {df['datetime'].values[-1]}",
#     yaxis=dict(),
#     yaxis2=dict(overlaying='y', side='right'),
#     height=650
# )

# Create the figure and add traces to it
fig = go.Figure(data=[trace1, trace2], layout=layout)

fig.update_xaxes(showspikes=True, spikemode="across", spikecolor="black", spikethickness=0.5)
fig.update_yaxes(showspikes=True, spikemode="across", spikecolor="black", spikethickness=0.5)

fig.write_image(
    f"img/backtesting {df['ticker'].values[0]} {df['datetime'].values[0]} - {df['datetime'].values[-1]}.png",
    width=1920,
    height=1080
)

# Render the figure
fig.show()

In [2786]:
trace_value = 'u_prob'

x=df.index.values

y1=df['close'].values
y2=df[trace_value].values

# Create the first trace with the primary y-axis
trace1 = go.Scatter(x=x, y=y1, name='Close')

# Create the second trace with the secondary y-axis
trace2 = go.Scatter(x=x, y=y2, name=f"{trace_value}", yaxis='y2')

# Create the layout with two y-axes
layout = go.Layout(
    title=f"{df['ticker'].values[0]} {df['datetime'].values[0]} - {df['datetime'].values[-1]}",
    yaxis=dict(),
    yaxis2=dict( overlaying='y', side='right'),
    height=650
)

# Create the figure and add traces to it
fig = go.Figure(data=[trace1, trace2], layout=layout)

fig.update_xaxes(showspikes=True, spikemode="across", spikecolor="black", spikethickness=0.5)
fig.update_yaxes(showspikes=True, spikemode="across", spikecolor="black", spikethickness=0.5)

In [2787]:
fig = make_subplots(rows=1, cols=1, specs=[[{}]], vertical_spacing=0)

# Plot closing prices
fig.add_trace(
    go.Scatter(x=df.index, y=df['close'], line=dict(color="blue", width=1),name="Close"), row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=df.index,
        y=df['l_bound'],
        line=dict(color="red", width=1),
        name='Lower Bound'
    ),
    row=1,
    col=1
)

fig.add_trace(
    go.Scatter(
        x=df.index,
        y=df['lbf'],
        line=dict(color="green", width=1),
        name='LBF'
    ),
    row=1,
    col=1
)

fig.add_trace(
    go.Scatter(
        x=df.index,
        y=df['u_bound'],
        line=dict(color="red", width=1),
        name='Upper Bound'
    ),
    row=1,
    col=1
)

#  Highest Score: {highest_score}

fig.update_layout(
    title={'text': f"{df['ticker'].values[0]} {df['datetime'].values[0]} - {df['datetime'].values[-1]}", 'x': 0.5},
    width=1000, height=500,
    xaxis={"rangeslider": {"visible": False}},
    plot_bgcolor="#EFF5F8"
)

fig.update_yaxes(visible=False, secondary_y=True)